# Convert csv to to geojson using geopandas
- [geopandas.org](https://geopandas.org/en/stable/index.html)

In [1]:
from pathlib import Path

import geopandas
import pandas as pd

In [2]:
DATA_PATH = Path().resolve().parent.parent.parent/"data"
print(DATA_PATH)

/home/pyuser/workspace/data


## read raw data from csv file

In [3]:
raw_cities = pd.read_csv(DATA_PATH/"datasets"/"simplemaps_worldcities.csv")
raw_cities.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140


## filter target data

In [4]:
cities = raw_cities[raw_cities["country"].isin(["Portugal", "Spain"])]
cities.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
163,Barcelona,Barcelona,41.3825,2.1769,Spain,ES,ESP,Catalonia,admin,4588000.0,1724594040
238,Madrid,Madrid,40.4189,-3.6919,Spain,ES,ESP,Madrid,primary,3266126.0,1724616994
586,Sevilla,Sevilla,37.3828,-5.9731,Spain,ES,ESP,Andalusia,admin,1212045.0,1724991838
748,Lisbon,Lisbon,38.7452,-9.1604,Portugal,PT,PRT,Lisboa,primary,506654.0,1620619017
968,Valencia,Valencia,39.4667,-0.3750,Spain,ES,ESP,Valencia,admin,794288.0,1724981666


## convert to geojson

### create POINT geometry from coordinates

In [5]:
geometry = geopandas.points_from_xy(cities.lng, cities.lat)
geometry

<GeometryArray>
[  <POINT (2.177 41.382)>,  <POINT (-3.692 40.419)>,  <POINT (-5.973 37.383)>,
   <POINT (-9.16 38.745)>,  <POINT (-0.375 39.467)>,  <POINT (-0.883 41.648)>,
  <POINT (-4.417 36.717)>,    <POINT (2.65 39.567)>,  <POINT (-1.142 37.987)>,
 <POINT (-15.431 28.127)>,
 ...
  <POINT (-2.385 43.307)>,  <POINT (-7.541 40.818)>,  <POINT (-3.737 36.735)>,
 <POINT (-28.208 38.682)>,  <POINT (-7.244 39.413)>,   <POINT (-8.225 37.65)>,
  <POINT (-9.375 38.851)>,  <POINT (-8.912 37.082)>,   <POINT (-7.733 40.85)>,
  <POINT (-7.676 38.953)>]
Length: 350, dtype: geometry

### create a geo DataFrame

In [6]:
geo_df = (
    geopandas.GeoDataFrame(
        cities.filter(
            ["city_ascii", "country", "iso2", "iso3", "admin_name", "capital", "population"]
        ),
        geometry=geometry
    )
    .astype({
        "city_ascii": "string",
        "country": "string",
        "iso2": "string",
        "iso3": "string",
        "admin_name": "string",
        "capital": "string",
        "population": "int32"
    })
)
geo_df.head()

,city_ascii,country,iso2,iso3,admin_name,capital,population,geometry
163,Barcelona,Spain,ES,ESP,Catalonia,admin,4588000,POINT (2.17690 41.38250)
238,Madrid,Spain,ES,ESP,Madrid,primary,3266126,POINT (-3.69190 40.41890)
586,Sevilla,Spain,ES,ESP,Andalusia,admin,1212045,POINT (-5.97310 37.38280)
748,Lisbon,Portugal,PT,PRT,Lisboa,primary,506654,POINT (-9.16040 38.74520)
968,Valencia,Spain,ES,ESP,Valencia,admin,794288,POINT (-0.37500 39.46670)


In [7]:
geo_df.to_file(DATA_PATH/"datasets"/"geojson"/"cities_pt_es.geojson", driver="GeoJSON")